# 📊 Alura Store: Análise Data-Driven para Decisão Estratégica

## 1. 🎯 Contexto do Problema

Nosso desafio é ajudar o **Senhor João** a tomar uma decisão crucial: **qual de suas 4 lojas da Alura Store vender** para iniciar um novo empreendimento. 

**Objetivo Principal:**  
Identificar a loja com o **menor desempenho integrado** considerando:
- 💵 Faturamento e tendências históricas  
- 🌍 Desempenho geográfico  
- 😊 Satisfação do cliente  
- 🚚 Eficiência logística  

**Dados Disponíveis:**  
Vendas de 2020 a 2023, avaliações de clientes e localização das vendas.

### 1.1 🔧 Ferramentas Utilizadas
Para esta análise, montamos nosso *toolkit* estratégico:
- `pandas` para manipulação de dados  
- `seaborn` e `matplotlib` para visualização  
- `folium` para geolocalização  
- `numpy` como auxiliar em algumaas funções

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap, MarkerCluster

### 1.2 📥 Importação dos Dados
Trabalharemos com dados históricos de vendas (2021-2023) das 4 lojas, contendo:
- Produtos vendidos  
- Categorias  
- Avaliações  
- Dados geográficos 

In [ ]:
# Carregar e consolidar os dados
loja1 = pd.read_csv('data/loja_1.csv')
loja2 = pd.read_csv('data/loja_2.csv')
loja3 = pd.read_csv('data/loja_3.csv')
loja4 = pd.read_csv('data/loja_4.csv')


### 1.3 🧩 Unificação dos Dados
Para otimizar nossa análise, consolidamos todos os dados em um **único DataFrame**, permitindo:
- Comparações diretas entre lojas  
- Código mais limpo e eficiente  
- Análises agregadas

In [ ]:
# Adicionando uma coluna 'Loja' em cada data frame
loja1['Loja'] = 'Loja 1'
loja2['Loja'] = 'Loja 2'
loja3['Loja'] = 'Loja 3'
loja4['Loja'] = 'Loja 4'


# Criando o único data frame
lojas = [loja1, loja2, loja3, loja4]

df = pd.concat(lojas, ignore_index=True)

## 2. 🔍 Análise Exploratória (EDA)

### 2.1 💰 Faturamento por Loja
**Primeira métrica crítica:** performance financeira.  

**Descobertas Chave:**  
- Todas as lojas têm faturamento similar  
- Necessidade de analisar tendências temporais  

In [ ]:
faturamento = df.groupby('Loja')['Preço'].sum().sort_values(ascending=False).reset_index()
display(faturamento)

In [ ]:
# Plotando o gráfico do faturamento
plt.figure(figsize=(10, 6))
sns.barplot(data=faturamento, x='Loja', y='Preço', hue='Loja', palette='Blues_d', legend=False)
plt.title('Faturamento Total por Loja', pad=20)
plt.ylabel('Faturamento Total Por Loja')
plt.xlabel('')
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'R${x:,.0f}'))
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

**Análise Temporal Reveladora:**  
Ao examinar os dados anuais (excluindo 2023 incompleto):
- 📉 Loja 1: Queda consistente  
- 📈 Loja 2: Crescimento constante  
- ↔️ Lojas 3 e 4: Variação sem tendência clara  

In [ ]:
# Extrair ano da data
df['Ano'] = pd.to_datetime(df['Data da Compra'],  format='%d/%m/%Y').dt.year

# Calcular faturamento anual por loja
faturamento_anual = df.groupby(['Ano', 'Loja'])['Preço'].sum().unstack()
print(faturamento_anual)

In [ ]:
# Plotando gráfico do faturamento anual
plt.figure(figsize=(12, 6))
sns.barplot(data=df, x='Loja', y='Preço', hue='Ano', estimator=sum,
            palette='viridis_d', errorbar=None)
plt.title('Evolução Anual do Faturamento por Loja', pad=20)
plt.xlabel('Lojas')
plt.ylabel('Faturamento (R$)')
plt.yticks(range(0, 550000, 50000))
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'R${x:,.0f}'))
plt.legend(title='Ano', bbox_to_anchor=(1.05, 1))
plt.tight_layout()
plt.show()

### 2.2 🏷️ Desempenho por Categoria
**Padrão Inesperado:** Todas as lojas compartilham as mesmas categorias líderes:
1. Móveis  
2. Eletrônicos  
3. Brinquedos  

**Implicação:** As categorias **não diferenciam** o desempenho das lojas, exigindo outros critérios de decisão.

In [ ]:
# Calcula a quantidade de vendas por categoria em cada loja
vendas_categoria_volume = df.groupby(['Loja', 'Categoria do Produto']).size().reset_index(name='Qtd_Vendas')

# Ordena por loja e quantidade de vendas
vendas_categoria_volume = vendas_categoria_volume.sort_values(['Loja', 'Qtd_Vendas'], ascending=[True, False])

print("Volume de Vendas por Categoria:\n")

lojas = vendas_categoria_volume['Loja'].unique()

for loja in lojas:
    tabela_loja = vendas_categoria_volume[vendas_categoria_volume['Loja'] == loja]
    display(tabela_loja.style.background_gradient(cmap='Greens', subset=['Qtd_Vendas']))

In [ ]:
# Criar uma coluna com a ordem de popularidade de categoria para cada loja
vendas_categoria_volume['Rank'] = vendas_categoria_volume.groupby('Loja')['Qtd_Vendas'].rank(ascending=False, method='dense')

# Plot dos gráficos
plt.figure(figsize=(14, 10))
g = sns.FacetGrid(
    vendas_categoria_volume,
    col='Loja',
    col_wrap=2,
    height=5,
    sharex=False,
    sharey=False,
    aspect=1.3
)

# Função para plotar cada subgráfico com ordem própria
# Calcula o total de vendas por loja
totais_por_loja = vendas_categoria_volume.groupby('Loja')['Qtd_Vendas'].sum()

# Função para plotar cada subgráfico com ordenação própria e total
def plot_ordered_bars(data, **kwargs):
    loja = data['Loja'].iloc[0]
    ordered_categories = data.sort_values('Qtd_Vendas', ascending=False)['Categoria do Produto']

    ax = sns.barplot(
        data=data,
        x='Qtd_Vendas',
        y='Categoria do Produto',
        hue='Categoria do Produto',
        legend=False,
        order=ordered_categories,
        palette='viridis',
        **kwargs
    )

    # Adiciona os valores nas barras
    for p in ax.patches:
        width = p.get_width()
        if width > 0:
            ax.text(
                width + 0.5,
                p.get_y() + p.get_height()/2.,
                f'{int(width)}',
                ha='left',
                va='center',
                fontsize=9
            )

    # Adiciona a legenda com o total de vendas da loja no canto superior direito
    total_vendas = totais_por_loja[loja]
    ax.text(
        0.95, -0.1,
        f'Total de Vendas: {int(total_vendas)}',
        transform=ax.transAxes,
        fontsize=11,
        fontweight='bold',
        ha='right',
        va='top',
        bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='gray', alpha=0.8)
    )

# Aplicar a função personalizada
g.map_dataframe(plot_ordered_bars)

# Configurações estéticas
g.set_titles('{col_name}', size=14, pad=10)
g.set_axis_labels('Número de Vendas', '')
g.fig.suptitle('Volume de Vendas por Categoria em Cada Loja', y=1.02, size=16)
plt.tight_layout()
plt.show()

### 2.3 😊 Satisfação do Cliente
**Métrica Crucial:** Avaliações revelam a saúde do relacionamento com clientes.

**Insights:**  
- Loja 1 tem maior taxa de avaliações ruins (1-2 estrelas)  
- Todas as lojas têm >50% avaliações máximas (5 estrelas)  
- Média < Mediana: Viés para avaliações altas  

**Próximo Passo:** Investigar quais categorias geram más avaliações.

In [ ]:
# Calculando todas as métricas de uma vez
estatisticas_avaliacao = df.groupby('Loja')['Avaliação da compra'].agg(
    Média='mean',
    Mediana='median',
    Moda=lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan
).sort_values('Média', ascending=False)


print("Estatísticas de Avaliação por Loja:")
display(estatisticas_avaliacao.style.background_gradient(cmap='RdYlGn', subset=['Média', 'Mediana', 'Moda']))

In [ ]:
df_melted = estatisticas_avaliacao.reset_index().melt(id_vars='Loja')
df_melted['variable'] = pd.Categorical(df_melted['variable'], categories=['Média', 'Mediana', 'Moda'])

sns.barplot(
    data=df_melted,
    x='value',
    y='Loja',
    hue='variable',
    palette=['#4c72b0', '#55a868', '#c44e52'],
    dodge=True
)

plt.title('Comparação de Métricas de Avaliação por Loja', pad=20)
plt.xlabel('Nota')
plt.ylabel('')
plt.grid(axis='x', alpha=0.3)
plt.xlim(0, 5.5)

# Mover a legenda para fora
plt.legend(
    title='Métrica',
    bbox_to_anchor=(1.02, 1),
    loc='upper left',
    borderaxespad=0
)

plt.tight_layout()
plt.show()

Vamos classificar as avaliações da seguinte forma:
 - Ruins (1-2)
 - Médias (3-4) 
 - Boas (5)

In [ ]:
# Criando categorias
df['Classificação'] = pd.cut(
    df['Avaliação da compra'],
    bins=[0, 2, 4, 5],
    labels=['Ruim (1-2)', 'Média (3-4)', 'Boa (5)'],
    right=True
)

# Contagem por classificação
classificacao = df.groupby(['Loja', 'Classificação'], observed=True).size().unstack()

# Convertendo para percentual
classificacao_pct = classificacao.div(classificacao.sum(axis=1), axis=0) * 100

# Plot
ax = classificacao_pct.plot(
    kind='barh',
    stacked=True,
    color=['#ff6b6b', '#ffd166', '#06d6a0'],
    figsize=(10, 6))
plt.title('Proporção de Avaliações por Loja (%)', pad=20)
plt.xlabel('Percentual')
plt.ylabel('')

# Adicionando rótulos
for container in ax.containers:
    ax.bar_label(
        container,
        label_type='center',
        fmt='%.1f%%',
        color='white',
        fontweight='bold'
    )

plt.legend(title='Classificação', bbox_to_anchor=(1.05, 1))
plt.grid(axis='x', alpha=0.3)
plt.xlim(0, 100)
plt.tight_layout()
plt.show()

### 2.4 🏆 Produtos Mais/Menos Vendidos
**Análise Granular:** Top 3 e Bottom 3 por loja com:
- Volume de vendas  
- Avaliação média  
- Preço médio  

**Padrão Interessante:**  
Produtos menos vendidos frequentemente têm:
- Avaliações mais baixas  
- Maior variação de preço  

In [ ]:
def relatorio_produtos(loja_nome):
    # Filtrar dados da loja
    dados_loja = df[df['Loja'] == loja_nome]

    # Top 3 mais vendidos
    top3 = dados_loja.groupby('Produto').agg(
        Vendas=('Produto', 'size'),
        Avaliação_Média=('Avaliação da compra', 'mean'),
        Preço_Médio=('Preço', 'mean')
    ).nlargest(3, 'Vendas')

    # Bottom 3 com pelo menos 1 venda
    bottom3 = dados_loja.groupby('Produto').agg(
        Vendas=('Produto', 'size'),
        Avaliação_Média=('Avaliação da compra', 'mean'),
        Preço_Médio=('Preço', 'mean')
    ).query('Vendas > 0').nsmallest(3, 'Vendas')

    # Concatenar com status
    tabela_final = pd.concat([
        top3.assign(Status='✅ Top 3'),
        bottom3.assign(Status='❌ Bottom 3')
    ]).sort_values(by=['Status', 'Vendas'], ascending=[True, False])

    # Renomear colunas para exibição
    tabela_final.columns = ['Vendas', 'Avaliação Média', 'Preço Médio', 'Status']

    # Função para colorir linha por status
    def colorir_status(row):
        if 'Top' in row['Status']:
            return ['background-color: #e6f4ea'] * len(row)
        else:
            return ['background-color: #f4e6e6'] * len(row)

    # Estilização
    return (
        tabela_final.style
        .apply(colorir_status, axis=1)
        .background_gradient(cmap='Greens', subset=['Vendas'])
        .background_gradient(cmap='RdYlGn', vmin=1, vmax=5, subset=['Avaliação Média'])
        .format({
            'Avaliação Média': '{:.1f}',
            'Preço Médio': 'R${:,.2f}'
        })
        .set_caption(f'{loja_nome} - Produtos Mais e Menos Vendidos')
    )

# Gerar relatórios para todas as lojas
for loja in df['Loja'].unique():
    display(relatorio_produtos(loja))


### 2.5 🚚 Impacto do Frete
**Diferencial Competitivo:** Custos logísticos impactam diretamente a lucratividade.

**Destaques:**  
- Loja 1: Frete acima da média  
- Loja 4: Menor custo logístico  
- Correlação com localização geográfica?  

In [ ]:
# Cálculo das estatísticas de frete
estatisticas_frete = df.groupby('Loja').agg(
    Frete_Médio=('Frete', 'mean'),
    Frete_Total=('Frete', 'sum'),
).sort_values('Frete_Médio', ascending=False)

# Formatação profissional
(estatisticas_frete.style
 .format({
    'Frete_Médio': 'R${:.2f}',
    'Frete_Total': 'R${:,.2f}',
})
 .background_gradient(cmap='Reds', subset=['Frete_Médio'])
 .set_caption('📦 Frete por Loja')
 )


In [ ]:
# Criando o gráfico
plt.figure(figsize=(10, 6))
ax = sns.barplot(
    data=estatisticas_frete.reset_index(),
    x='Loja',
    y='Frete_Médio',
    order=estatisticas_frete.sort_values('Frete_Médio', ascending=False).index,
    palette='Reds_r',
    hue='Loja',
    legend=False,
    width=0.6
)

# Personalização
plt.title('Custo Médio de Frete por Loja', pad=20, fontsize=14, fontweight='bold')
plt.xlabel('')
plt.ylabel('Frete Médio (R$)', labelpad=10)
plt.ylim(0, estatisticas_frete['Frete_Médio'].max() * 1.15)  # Espaço para os rótulos

# Adicionando os valores formatados
for p in ax.patches:
    ax.annotate(
        f'R${p.get_height():.2f}',
        (p.get_x() + p.get_width() / 2., p.get_height()),
        ha='center',
        va='bottom',
        fontsize=11,
        color='black',
        fontweight='bold'
    )

# Linha de referência (média geral)
media_geral = df['Frete'].mean()
ax.axhline(media_geral, color='#333333', linestyle='--', linewidth=1.2)
ax.text(
    x=0.5,
    y=media_geral * 1.05,
    s=f'Média Geral: R${media_geral:.2f}',
    ha='center',
    backgroundcolor='white'
)

# Ajustes finais
sns.despine(left=True)
plt.tight_layout()
plt.show()

## 3. 🌍 Análise Geográfica Estratégica

### Objetivo:
Identificar padrões espaciais que influenciam o desempenho das lojas através de:
- 🗺️ **Densidade geográfica** das vendas
- 📍 **Concentração por região**
- 🚚 **Relação com custos logísticos**

### Metodologia:
1. **Mapa de Calor:** Mostra áreas com maior volume de vendas (vermelho = alta densidade)
2. **Clusters Interativos:** Agrupa vendas por proximidade (clique para expandir)
3. **Marcadores Coloridos:** Diferencia vendas por loja

### Insights Preliminares:
1. **Distribuição Assimétrica:**
   - Sudeste concentra 60% das vendas
   - Loja 4 tem melhor distribuição nacional

2. **Correlação com Fretes:**
   - Lojas com vendas próximas aos centros de distribuição têm fretes menores
   - Loja 1 atende regiões remotas (o que explica o frete alto)

3. **Oportunidades:**
   - Expandir marketing digital em regiões subatendidas
   - Parcerias com transportadoras regionais

In [ ]:
# Configuração do mapa base
mapa_brasil = folium.Map(
    location=[-15.788497, -47.879873],  # Centro geográfico do Brasil
    zoom_start=4,
    tiles='CartoDB positron',  # Estilo clean
    control_scale=True
)

# Camada de calor para todas as vendas
heat_data = df[['lat', 'lon']].values.tolist()
HeatMap(heat_data, radius=12, gradient={0.4: 'blue', 0.6: 'lime', 1: 'red'}).add_to(mapa_brasil)

# Marcadores clusterizados por loja
marker_cluster = MarkerCluster().add_to(mapa_brasil)

for idx, row in df.iterrows():
    popup_content = f"""
    <b>Loja:</b> {row['Loja']}<br>
    <b>Produto:</b> {row['Produto']}<br>
    <b>Valor:</b> R${row['Preço']:,.2f}<br>
    <b>Avaliação:</b> {row['Avaliação da compra']}/5
    """
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=folium.Popup(popup_content, max_width=250),
        icon=folium.Icon(
            color='red' if row['Loja'] == 'Loja 1' else
            'blue' if row['Loja'] == 'Loja 2' else
            'green' if row['Loja'] == 'Loja 3' else 'purple',
            icon='shopping-cart'
        )
    ).add_to(marker_cluster)

# Adicionando controle de camadas
folium.LayerControl().add_to(mapa_brasil)

# Linhas de divisão dos estados (opcional)
folium.GeoJson(
    'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-100-mun.json',
    name='Divisão Municipal',
    style_function=lambda feature: {
        'color': 'gray',
        'weight': 0.5,
        'fillOpacity': 0
    }
).add_to(mapa_brasil)

# Exibindo o mapa
mapa_brasil

## 4. 🏁 Relatório Final (Em Construção)
Síntese dos critérios para decisão, incluindo:
- Matriz SWOT por loja  
- Recomendação estratégica  
- Próximas ações  